## Importacion de modulos necesarios

In [4]:
import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt

## Configuracion de variables necesarias

In [152]:
CLIENTS = ['Carlos', 'Raul', 'Alejandro', 'Sofia', 'Laura', 'Juan', 'Jose', 'Pedro', 'Sebastian', 'Maria', 'Aitana', 'Dana', 'Betzie', 'Monica', 'Rebecca', 'Pepe', 'Lucas', 'Rubio']
RESTAURANTS = ['Restaurante-1', 'Restaurante-2', 'Restaurante-3', 'Restaurante-4', 'Restaurante 5']
DELIVERY_MEN = ['Daniela', 'David', 'Mario', 'Roberto', 'Rony', 'Alberto', 'Joaquin']

TOTAL_DELIVERY_MEN = len(DELIVERY_MEN)
TOTAL_CLIENTS = len(CLIENTS)
TOTAL_RESTAURANTS = len(RESTAURANTS)

## Helpers

In [155]:
def delivery_availability(env, delivery_men):
    while True:
        # Tiempo aleatorio de espera para disponibilidad de repartidor
        yield env.timeout(random.randint(20, 60)) # 20min y 1h

## Inicializacion de dataframes

In [158]:
df_order = pd.DataFrame(columns=['id', 'time', 'client', 'restaurant', 'delivery', 'estimated_time', 'real_time_deliver', 'state'])

## Codigo principal

In [161]:
class Order:
    def __init__(self, env, id, client, restaurant):
        self.env = env
        self.client = client
        self.id = id
        self.restaurant = restaurant
        self.preparation_time = round(15 / self.restaurant.num_of_chefs)
        self.delivery_time = random.randint(10, 20)
        self.max_waiting_time = random.randint(15, 40)

In [163]:
class Client:
    def __init__(self, id, name):
        self.id = id
        self.name = name

In [165]:
class Delivery:
    def __init__(self, id, name):
        self.id = id
        self.name = name

In [167]:
class Restaurant:
    def __init__(self, env, id, name, num_of_cashiers, num_of_chefs):
        self.id = id
        self.name = name
        self.num_of_cashiers = num_of_cashiers
        self.env = env
        self.num_of_chefs = num_of_chefs
        self.cashiers = simpy.Resource(self.env, capacity=num_of_cashiers)
        self.chefs = simpy.Resource(self.env, capacity=num_of_chefs)

In [169]:
class Main: 
    def __init__(self, env, clients, restaurants, delivery_men):
        self.env = env
        self.clients = clients
        self.restaurant = restaurants
        self.delivery_men = delivery_men
        self.list_of_busy_delivery_men = [] 

    def order_arrival(self, interval):
        client_id = 0
        while True:
            yield self.env.timeout(random.expovariate(1.0 / interval))
            client = random.choice(self.clients)
            restaurant = random.choice(self.restaurant)
            client_id += 1
            print(f'{round(self.env.now)}: Nuevo pedido realizado por: {client.name} en el restaurante: {restaurant.name}')
            self.env.process(self.process_order(client, restaurant))
    
    def process_order(self, client, restaurant):
        initial_time = self.env.now
        order = Order(self.env, random.sample(list(range(1, 1000000)), 1)[0], client, restaurant)
        time_arrives_order = self.env.now

        with self.delivery_men.request() as delivery_req:
            yield delivery_req
            global df_events  
            global df_restaurant_data
            
            # [Daniela, Mario, David] --> Todos los deliveries
            # [Daniela] -- Deliveries ocupados
            delivery = random.choice(list(set(delivery_arr) - set(self.list_of_busy_delivery_men)))
            self.list_of_busy_delivery_men.append(delivery)
            
            print(f'{round(self.env.now)}: Delivery {delivery.name} toma la orden del cliente: {client.name} y va hacia el restaurante: {restaurant.name}')
            time_order_taken = round(self.env.now - time_arrives_order)
            time_afer_take_order = self.env.now

            df_temp_event = pd.DataFrame({
                'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                'order': order.id,
                'order_client_id': order.client.id,
                'order_client_name': order.client.name,
                'time': round(self.env.now),
                'delivery': delivery.name,
                'time_per_event': time_order_taken,
                'description_event': 'Orden tomada por delivery'
            }, index=[0 if df_events.empty else df_events.index[-1] + 1])

            df_events = pd.concat([df_events, df_temp_event])

            # Tiempo en llegar al restaurante
            yield self.env.timeout(random.randint(2, 7))
            time_to_restaurant = round(self.env.now - time_afer_take_order)

            df_temp_event = pd.DataFrame({
                'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                'order': order.id,
                'order_client_id': order.client.id,
                'order_client_name': order.client.name,
                'time': round(self.env.now),
                'delivery': delivery.name,
                'time_per_event': time_to_restaurant,
                'description_event': 'En camino al restaurante'
            }, index=[0 if df_events.empty else df_events.index[-1] + 1])

            df_events = pd.concat([df_events, df_temp_event])
                
            with restaurant.cashiers.request() as cashier_req:
                yield cashier_req
                print(f'{round(self.env.now)}: Pedido del cliente {client.name} esta siendo atendido en caja en el restaurante: {restaurant.name}')
                time_at_cashier = round(self.env.now)
                
                # Tiempo de espera en caja
                yield self.env.timeout(random.randint(1, 3))
                time_after_cashier = round(self.env.now - time_at_cashier)

                df_temp_event = pd.DataFrame({
                    'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                    'order': order.id,
                    'order_client_id': order.client.id,
                    'order_client_name': order.client.name,
                    'time': round(time_at_cashier),
                    'delivery': delivery.name,
                    'time_per_event': time_after_cashier,
                    'description_event': 'En caja'
                }, index=[0 if df_events.empty else df_events.index[-1] + 1])
        
                df_events = pd.concat([df_events, df_temp_event])
            
            with restaurant.chefs.request() as chef_req:
                yield chef_req
                print(f'{round(self.env.now)}: Pedido del cliente {client.name} en preparacion en el restaurante: {restaurant.name}')
                time_start_waiting_order = round(self.env.now)
                
                yield self.env.timeout(order.preparation_time)
                time_after_waiting_order = round(self.env.now) - time_start_waiting_order
                print(f'{round(self.env.now)}: Orden del cliente {client.name} entregada al repartidor {delivery.name} en el restaurante: {restaurant.name}')

                df_temp_event = pd.DataFrame({
                    'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                    'order': order.id,
                    'order_client_id': order.client.id,
                    'order_client_name': order.client.name,
                    'time': round(time_start_waiting_order),
                    'delivery': delivery.name,
                    'time_per_event': time_after_waiting_order,
                    'description_event': 'Esperando orden'
                }, index=[0 if df_events.empty else df_events.index[-1] + 1])
        
                df_events = pd.concat([df_events, df_temp_event])

                df_temp_restaurant_data = pd.DataFrame({
                    'id': restaurant.id,
                    'name': restaurant.name,
                    'order': order.id,
                    'num_of_cashiers': restaurant.num_of_cashiers,
                    'num_of_chefs': restaurant.num_of_chefs,
                    'time_in_cashier': time_after_cashier,
                    'time_in_kitchen': time_after_waiting_order,
                    'total_time_in_restaurant': time_after_cashier + time_after_waiting_order
                }, index=[0 if df_restaurant_data.empty else df_restaurant_data.index[-1] + 1])

                df_restaurant_data = pd.concat([df_restaurant_data, df_temp_restaurant_data])
                
                time_go_to_client = self.env.now
            
            yield self.env.timeout(order.delivery_time)
            total_time = self.env.now - initial_time
            time_order_delivered = round(self.env.now - time_go_to_client)
            print(f'{round(self.env.now)}: Pedido del cliente: {client.name} entregado por: {delivery.name} en un total de: {round(total_time)}min. Estimado de entrega: {order.max_waiting_time}min')

            df_temp_event = pd.DataFrame({
                'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                'order': order.id,
                'order_client_id': order.client.id,
                'order_client_name': order.client.name,
                'time': round(time_go_to_client),
                'delivery': delivery.name,
                'time_per_event': time_order_delivered,
                'description_event': 'Orden entregada'
            }, index=[0 if df_events.empty else df_events.index[-1] + 1])
        
            df_events = pd.concat([df_events, df_temp_event])
            
            self.list_of_busy_delivery_men.remove(delivery)

            global df_order  
            df_temp_order = pd.DataFrame({
                'id': order.id,
                'time': round(initial_time),
                'client': client.name,
                'restaurant': restaurant.name,
                'delivery': delivery.name,
                'estimated_time': order.max_waiting_time,
                'real_time_deliver': round(total_time),
                'state': 'A tiempo' if round(total_time) <= order.max_waiting_time else 'Fuera de tiempo'
            }, index=[0 if df_order.empty else df_order.index[-1] + 1])

            df_order = pd.concat([df_order, df_temp_order])
            print('-' * 100)

In [171]:
env = simpy.Environment()

# VARIABLES NECESARIAS
delivery_men = simpy.Resource(env, capacity=TOTAL_DELIVERY_MEN)
clients = [Client(i, name) for i, name in enumerate(CLIENTS)]
restaurants = [Restaurant(env, i, name, random.randint(1, 3), random.randint(2, 5)) for i, name in enumerate(RESTAURANTS)]
delivery_arr = [Delivery(i, name) for i, name in enumerate(DELIVERY_MEN)]

# Inicializacion de dataframes
clients_data = [{'id': client.id, 'name': client.name} for client in clients]
df_client = pd.DataFrame(clients_data)

restaurants_data = [{'id': restaurant.id, 'name': restaurant.name, 'num_of_cashiers': restaurant.num_of_cashiers, 'num_of_chefs': restaurant.num_of_chefs} for restaurant in restaurants]
df_restaurant = pd.DataFrame(restaurants_data)

delivery_data = [{'id': delivery.id, 'name': delivery.name} for delivery in delivery_arr]
df_delivery = pd.DataFrame(delivery_data)

df_order = pd.DataFrame(columns=['id', 'time', 'client', 'restaurant', 'delivery', 'estimated_time', 'real_time_deliver', 'state'])
df_restaurant_data = pd.DataFrame(columns=['id', 'name', 'order', 'num_of_cashiers', 'num_of_chefs', 'time_in_cashier', 'time_in_kitchen', 'total_time_in_restaurant'])
df_events = pd.DataFrame(columns=['id', 'order', 'order_client_id', 'order_client_name', 'time', 'delivery', 'time_per_event', 'description_event'])

# instancia de la clase main
app = Main(env, clients, restaurants, delivery_men)

# PROCESOS
env.process(app.order_arrival(10))
env.process(delivery_availability(env, delivery_men))

# Ejecutar la simulacion
env.run(5000)

4: Nuevo pedido realizado por: Sebastian en el restaurante: Restaurante-4
4: Delivery Daniela toma la orden del cliente: Sebastian y va hacia el restaurante: Restaurante-4
8: Pedido del cliente Sebastian esta siendo atendido en caja en el restaurante: Restaurante-4
10: Nuevo pedido realizado por: Alejandro en el restaurante: Restaurante-3
10: Delivery Alberto toma la orden del cliente: Alejandro y va hacia el restaurante: Restaurante-3
11: Pedido del cliente Sebastian en preparacion en el restaurante: Restaurante-4
15: Orden del cliente Sebastian entregada al repartidor Daniela en el restaurante: Restaurante-4
15: Pedido del cliente Alejandro esta siendo atendido en caja en el restaurante: Restaurante-3
16: Pedido del cliente Alejandro en preparacion en el restaurante: Restaurante-3
19: Nuevo pedido realizado por: Jose en el restaurante: Restaurante-3
19: Delivery Rony toma la orden del cliente: Jose y va hacia el restaurante: Restaurante-3
19: Orden del cliente Alejandro entregada al 

In [173]:
df_restaurant_data

,id,name,order,num_of_cashiers,num_of_chefs,time_in_cashier,time_in_kitchen,total_time_in_restaurant
0,3,Restaurante-4,156682,1,4,3,4,7
1,2,Restaurante-3,683889,1,5,1,3,4
2,2,Restaurante-3,358819,1,5,3,3,6
3,3,Restaurante-4,378213,1,4,2,4,6
4,4,Restaurante 5,102764,2,4,2,4,6
...,...,...,...,...,...,...,...,...
540,0,Restaurante-1,536032,1,2,3,8,11
541,1,Restaurante-2,224467,1,4,2,4,6
542,2,Restaurante-3,17772,1,5,3,3,6
543,2,Restaurante-3,13787,1,5,2,3,5


In [175]:
df_order

,id,time,client,restaurant,delivery,estimated_time,real_time_deliver,state
0,683889,10,Alejandro,Restaurante-3,Alberto,15,19,Fuera de tiempo
1,156682,4,Sebastian,Restaurante-4,Daniela,23,26,Fuera de tiempo
2,358819,19,Jose,Restaurante-3,Rony,27,20,A tiempo
3,378213,31,Maria,Restaurante-4,Alberto,25,21,A tiempo
4,102764,32,Lucas,Restaurante 5,David,30,25,A tiempo
...,...,...,...,...,...,...,...,...
540,703091,4916,Juan,Restaurante-3,Joaquin,32,27,A tiempo
541,224467,4921,Pedro,Restaurante-2,Rony,28,23,A tiempo
542,17772,4923,Monica,Restaurante-3,Daniela,28,27,A tiempo
543,13787,4935,Raul,Restaurante-3,David,31,20,A tiempo


In [177]:
df_events

,id,order,order_client_id,order_client_name,time,delivery,time_per_event,description_event
0,1,156682,8,Sebastian,4,Daniela,0,Orden tomada por delivery
1,1,156682,8,Sebastian,8,Daniela,4,En camino al restaurante
2,2,683889,2,Alejandro,10,Alberto,0,Orden tomada por delivery
3,3,156682,8,Sebastian,8,Daniela,3,En caja
4,4,156682,8,Sebastian,11,Daniela,4,Esperando orden
...,...,...,...,...,...,...,...,...
2723,2723,439242,1,Raul,4953,Mario,8,Esperando orden
2724,2724,439242,1,Raul,4961,Mario,11,Orden entregada
2725,2725,242221,3,Sofia,4988,Daniela,0,Orden tomada por delivery
2726,2726,242221,3,Sofia,4990,Daniela,2,En camino al restaurante


## 1. Analisis entregas a tiempo y fuera de tiempo por Delivery
**¿Cuantas entregas se relizaron a tiempo y cuantas se realizaron fuera de tiempo?** (Esto mediria que tan bueno es el rendimiento de un repartidor)

In [180]:
df_order.groupby(['delivery', 'state']).size().unstack(fill_value=0)

state,A tiempo,Fuera de tiempo
delivery,,
Alberto,42,34
Daniela,51,29
David,40,34
Joaquin,40,36
Mario,59,25
Roberto,42,33
Rony,49,31


## 2. Analisis de Cuantas ordenes pidio cada cliente ordenadas del cliente que mas pidio al que menos pidio
**¿Quienes fueron los clientes que mas ordenes realizaron?** (Esto serviria para poder observar la fidelidad de los clientes)

In [183]:
df_order.groupby(['client']).size().sort_values(ascending=False).reset_index(name='Number of orders')

,client,Number of orders
0,Rubio,48
1,Pedro,38
2,Juan,36
3,Carlos,35
4,Lucas,34
5,Sofia,33
6,Raul,31
7,Alejandro,31
8,Aitana,31
9,Rebecca,30


## 3. Analisis de cantidad de ordenes atendidas por restaurante
**¿Cuantas ordenes fueron atendidas por restaurante y cuales restaurantes atendieron mas ordenes?** (Esto serviria para medir el rendimiento de cada restaurante)

In [186]:
df_order.groupby(['restaurant']).size().sort_values(ascending=False).reset_index(name='Number of orders')

,restaurant,Number of orders
0,Restaurante 5,117
1,Restaurante-2,117
2,Restaurante-3,111
3,Restaurante-4,102
4,Restaurante-1,98


## 4. Analis de tiempo promedio de entrega por delivery
**¿Cual es el tiempo promedio total de un pedido por delivery?** (Esto ayudaria a poder observar la velocidad con la que el repartidor realiza las entregas)

In [189]:
df_order.groupby('delivery')['real_time_deliver'].mean().reset_index().sort_values(by='real_time_deliver', ascending=False)

,delivery,real_time_deliver
6,Rony,26.575
5,Roberto,26.24
1,Daniela,25.8875
4,Mario,25.857143
2,David,25.743243
0,Alberto,25.723684
3,Joaquin,25.223684


## 5. Analisis de una orden en especifico
**Ver el log (Historial de tiempo cronologico) de una orden en especifico por evento** (Esto sirve para llevar un log/historial cronologico del tiempo de una orden en especifico por evento)

In [235]:
order_id = 3893
df_analysis_by_order = df_events[df_events['order'] == order_id]
df_analysis_by_order.sort_values('time')
df_analysis_by_order[['time', 'time_per_event', 'description_event']]

,time,time_per_event,description_event
2281,4132,0,Orden tomada por delivery
2282,4135,3,En camino al restaurante
2284,4135,1,En caja
2288,4136,4,Esperando orden
2298,4140,20,Orden entregada


## 6. Analis del tiempo total tomado por orden/pedido
**¿Cual es el tiempo total tomado de entrega por orden?** (Esto sirve para ver cual es el tiempo total en minutos que tardo una orden en ser entregada)

In [195]:
df_events.groupby('order')['time_per_event'].sum().reset_index()

,order,time_per_event
0,3893,28
1,4547,29
2,5795,22
3,7210,31
4,9004,34
...,...,...
541,989209,21
542,989805,29
543,994952,28
544,996109,28


## 7. Analisis de la media de los eventos que mas tiempo toman
**¿Cual es la media del tiempo por evento?** (Esto sirve para poder observar cual es el evento que mas tiempo toma y poder mejorar el rendimiento en ese aspecto en especifico)

In [198]:
df_events.groupby('description_event')['time_per_event'].mean().sort_values(ascending=False).reset_index()

,description_event,time_per_event
0,Orden entregada,14.721101
1,Esperando orden,4.515596
2,En camino al restaurante,4.514652
3,En caja,2.021978
4,Orden tomada por delivery,0.086081


## 8. Analisis tiempo real vs tiempo estimado
**¿Hubo rendimiento positivo entre el tiempo real de entrega con respecto al tiempo estimado? (En caso de ser negativo quiere decir que en total, la media del tiempo real de entregas fue bajo con respecto al tiempo estimado de entregas)** (Esto sirve para ver la media de la diferencia entre tiempo real y tiempo estimado)

In [201]:
df_order['time_difference'] = df_order['real_time_deliver'] - df_order['estimated_time']
df_order['time_difference'].mean()

-1.363302752293578

## 9. Analisis de la media de la diferencia de tiempo entre tiempo real de entrega y tiempo estimado de entrega
**¿Cual fue el rendimiento de cada repartidor con respecto al tiempo estimado y tiempo real de entrega?** (Esto ayuda a ver si la media del rendimiento por tiempo real y tiempo estimado por repartidor es positivo o no)

In [204]:
df_order.groupby('delivery')['time_difference'].mean().reset_index().sort_values(by='time_difference', ascending=False)

,delivery,time_difference
6,Rony,0.05
3,Joaquin,-0.605263
2,David,-1.243243
1,Daniela,-1.525
0,Alberto,-1.657895
5,Roberto,-1.76
4,Mario,-2.72619


## 10. Analisis de la media del tiempo total de atencion por restaurante
**¿Cuales fueron los restaurantes que mejor promedio en tiempo de atencion tuvieron?** (Medir rendimiento por restaurante en terminos de tiempo total en ser atendido)

In [207]:
df_restaurant_data.groupby('name')['total_time_in_restaurant'].mean().reset_index().sort_values(by='total_time_in_restaurant', ascending=False)

,name,total_time_in_restaurant
1,Restaurante-1,10.142857
2,Restaurante-2,6.051282
0,Restaurante 5,5.974359
4,Restaurante-4,5.921569
3,Restaurante-3,5.018018
